<a href="https://colab.research.google.com/github/dkadiry/BresciaNortonClassifier/blob/main/Booking_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import tensorflow as tf
import sklearn.linear_model as skl
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn import svm
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, mutual_info_regression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion
import xgboost as xgb
import time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint, uniform
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten
#from scikeras.wrappers import 
from keras.wrappers.scikit_learn import KerasClassifier


import warnings 
warnings.filterwarnings('ignore')


np.random.seed(42)


In [ ]:
# Useful Functions
# Function to convert labels to number
def strtobin(x):
    if x == 'Canceled':
      x = 1
    elif x == 'Not_Canceled':
      x = 0
    return x

# Simple Preprocessing function
def myPreProc(data):
   #write function
   procData = data

   # Create a Full Date Column so we can order the rows by date - helps the RNN learn
   procData = procData.rename(columns={'ArrivalYear': 'year', 'ArrivalMonth': 'month', 'ArrivalDate': 'day'})
   cols=["year","month","day"]
   procData['Fulldate'] = procData[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
   procData['Fulldate']=pd.to_datetime(procData['Fulldate'], errors = 'coerce')

   # Set invalid dates to NaT
   procData.loc[procData['Fulldate'].isnull(), 'Fulldate'] = pd.NaT

   #Remove any empty columns
   procData = procData.dropna(axis=1, how='all')

   #Drop rows with any missing data
   procData = procData.dropna(axis=0)

   # Encode categorical variables using one-hot encoding
   procData = pd.get_dummies(procData, columns=['RoomType', 'MealPlan', 'MarketSegment'])

   # Sort the data based on the Full Date column in ascending order
   #procData = procData.sort_values('Fulldate', ascending=True)
  
   return procData



In [ ]:
#Load the dataset 
pathName = "/content/drive/MyDrive/Hackathon/DataQuestDataset.csv"
myData = pd.read_csv(pathName, index_col="BookingID")

df_copy = myData.copy()

In [ ]:
#Preprocess Data
df_copy = myPreProc(df_copy)
df_copy['BookingStatus'] = df_copy['BookingStatus'].map(strtobin)

In [ ]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28988 entries, 1 to 29020
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   LeadTime                     28988 non-null  int64         
 1   year                         28988 non-null  int64         
 2   month                        28988 non-null  int64         
 3   day                          28988 non-null  int64         
 4   NumWeekendNights             28988 non-null  int64         
 5   NumWeekNights                28988 non-null  int64         
 6   Parking                      28988 non-null  int64         
 7   NumAdults                    28988 non-null  int64         
 8   NumChildren                  28988 non-null  int64         
 9   RepeatedGuest                28988 non-null  int64         
 10  NumPrevCancellations         28988 non-null  int64         
 11  NumPreviousNonCancelled      28988 non-nu

In [ ]:
# Extract Target Variable and Features
X = df_copy.drop(['Fulldate', 'BookingStatus'], axis=1)
Y = df_copy['BookingStatus']
print(X.info())

#Split the data using train_test-split
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.30, random_state=42)

# Normalize Xtrain data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)

X_train_categorical = X_train.iloc[:, 14:30] #Collect all Categorical training data
X_train_numerical = X_train.iloc[:, 0:14] # Collect all numerical training data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28988 entries, 1 to 29020
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   LeadTime                     28988 non-null  int64  
 1   year                         28988 non-null  int64  
 2   month                        28988 non-null  int64  
 3   day                          28988 non-null  int64  
 4   NumWeekendNights             28988 non-null  int64  
 5   NumWeekNights                28988 non-null  int64  
 6   Parking                      28988 non-null  int64  
 7   NumAdults                    28988 non-null  int64  
 8   NumChildren                  28988 non-null  int64  
 9   RepeatedGuest                28988 non-null  int64  
 10  NumPrevCancellations         28988 non-null  int64  
 11  NumPreviousNonCancelled      28988 non-null  int64  
 12  AvgRoomPrice                 28988 non-null  float64
 13  SpecialRequests 

In [ ]:
np.random.seed(42)
# Use SelectKBest to select the top 10 features based on mutual information function classification (handles categorical data)
selector_mfc = SelectKBest(score_func=mutual_info_classif, k=5)
X_new = selector_mfc.fit_transform(X_train_categorical, Y_train)

# Get the names of the selected features
feature_names = X_train_categorical.columns[selector_mfc.get_support()]
print(feature_names)

# Create a DataFrame with feature names and scores
feature_df_mfc = pd.DataFrame({'Feature': X_train_categorical.columns, 'Score': selector_mfc.scores_})

# Sort the DataFrame by feature scores in descending order
feature_df_mfc = feature_df_mfc.sort_values('Score', ascending=False)

# Get the names of the selected features in the order of the sorted DataFrame
mfc_selected_features = feature_df_mfc['Feature'][:5].tolist()
print(feature_df_mfc)
print(f"The top 5 categorical features using Mutual Information function are:\n {mfc_selected_features}")

Index(['RoomType_Room_Type 1', 'MealPlan_Meal Plan 1', 'MealPlan_Meal Plan 2',
       'MarketSegment_Complementary', 'MarketSegment_Corporate'],
      dtype='object')
                        Feature     Score
13      MarketSegment_Corporate  0.008039
8          MealPlan_Meal Plan 2  0.007970
7          MealPlan_Meal Plan 1  0.005248
12  MarketSegment_Complementary  0.005116
0          RoomType_Room_Type 1  0.004505
15         MarketSegment_Online  0.004379
10        MealPlan_Not Selected  0.004038
14        MarketSegment_Offline  0.003459
5          RoomType_Room_Type 6  0.003438
9          MealPlan_Meal Plan 3  0.001153
1          RoomType_Room_Type 2  0.000000
2          RoomType_Room_Type 3  0.000000
3          RoomType_Room_Type 4  0.000000
4          RoomType_Room_Type 5  0.000000
6          RoomType_Room_Type 7  0.000000
11       MarketSegment_Aviation  0.000000
The top 5 categorical features using Mutual Information function are:
 ['MarketSegment_Corporate', 'MealPlan_Meal Plan 

In [ ]:
# Use SelectKBest to select the top 5 features based on mutual information function regressor (Handles non categorical forms of data)
np.random.seed(42)
selector_mutual = SelectKBest(score_func=mutual_info_regression, k=5)
X_mutual = selector_mutual.fit_transform(X_train_numerical, Y_train)

# Create a DataFrame with feature names and scores
feature_df_mfr = pd.DataFrame({'Feature': X_train_numerical.columns, 'Score': selector_mutual.scores_})

# Sort the DataFrame by feature scores in descending order
feature_df_mfr = feature_df_mfr.sort_values('Score', ascending=False)

# Get the names of the selected features in the order of the sorted DataFrame
mutual_selected_features = feature_df_mfr['Feature'][:5].tolist()

print(feature_df_mfr)
print(f"The top features using Mutual Scoring function are:\n {mutual_selected_features}")

                    Feature     Score
0                  LeadTime  0.135769
12             AvgRoomPrice  0.089087
13          SpecialRequests  0.031001
2                     month  0.020520
6                   Parking  0.019852
1                      year  0.017942
9             RepeatedGuest  0.010373
5             NumWeekNights  0.008240
10     NumPrevCancellations  0.006534
8               NumChildren  0.005395
4          NumWeekendNights  0.004551
11  NumPreviousNonCancelled  0.001647
3                       day  0.000000
7                 NumAdults  0.000000
The top features using Mutual Scoring function are:
 ['LeadTime', 'AvgRoomPrice', 'SpecialRequests', 'month', 'Parking']


In [ ]:
# Concatenate the two dataframes
feature_scores = pd.concat([feature_df_mfc, feature_df_mfr])

# Sort the concatenated dataframe by scores in descending order
feature_scores = feature_scores.sort_values('Score', ascending=False)

# Get the names of the selected features in the order of the sorted DataFrame
selected_features = feature_scores['Feature'][:18].tolist()

print(feature_scores)

print(f"The top 10 features using both scoring functions are:\n {selected_features}")

                        Feature     Score
0                      LeadTime  0.135769
12                 AvgRoomPrice  0.089087
13              SpecialRequests  0.031001
2                         month  0.020520
6                       Parking  0.019852
1                          year  0.017942
9                 RepeatedGuest  0.010373
5                 NumWeekNights  0.008240
13      MarketSegment_Corporate  0.008039
8          MealPlan_Meal Plan 2  0.007970
10         NumPrevCancellations  0.006534
8                   NumChildren  0.005395
7          MealPlan_Meal Plan 1  0.005248
12  MarketSegment_Complementary  0.005116
4              NumWeekendNights  0.004551
0          RoomType_Room_Type 1  0.004505
15         MarketSegment_Online  0.004379
10        MealPlan_Not Selected  0.004038
14        MarketSegment_Offline  0.003459
5          RoomType_Room_Type 6  0.003438
11      NumPreviousNonCancelled  0.001647
9          MealPlan_Meal Plan 3  0.001153
3                           day  0

,Feature,Score
0,LeadTime,0.135769
1,AvgRoomPrice,0.089087
2,SpecialRequests,0.031001
3,month,0.020520
4,Parking,0.019852
5,year,0.017942
6,RepeatedGuest,0.010373
7,NumWeekNights,0.008240
8,MarketSegment_Corporate,0.008039
9,MealPlan_Meal Plan 2,0.007970


The top 10 features using both scoring functions are:
 ['LeadTime', 'AvgRoomPrice', 'SpecialRequests', 'month', 'Parking', 'year', 'RepeatedGuest', 'NumWeekNights', 'MarketSegment_Corporate', 'MealPlan_Meal Plan 2', 'NumPrevCancellations', 'NumChildren', 'MealPlan_Meal Plan 1', 'MarketSegment_Complementary', 'NumWeekendNights', 'RoomType_Room_Type 1', 'MarketSegment_Online', 'MealPlan_Not Selected']


In [ ]:
# Modify X to only include top_k_features
X_best = X.loc[:, selected_features]

# Split the data again using train_test_split with the modified X
X_train_best, X_val_best, Y_train_best, Y_val_best = train_test_split(X_best, Y, test_size=0.30, random_state=42)

Using XGBoost

In [ ]:
#Untuned XGB model with best 10 features
xgb_untuned = xgb.XGBClassifier(seed=42)
xgb_untuned.fit(X_train_best, Y_train_best)
xgb_untuned_pred = xgb_untuned.predict(X_val_best)

validation_accuracy = accuracy_score(Y_val_best, xgb_untuned_pred)
training_accuracy = accuracy_score(Y_train_best, xgb_untuned.predict(X_train_best))

print("XGBoost Untuned Training Accuracy score with top 18 features score::", training_accuracy)
print("XGBoost Untuned Validation Accuracy score with top 18 features score:", validation_accuracy)

#XGB model with all features
xgb_default = xgb.XGBClassifier(seed=42)
xgb_default.fit(X_train, Y_train)
xgb_default_pred = xgb_default.predict(X_val)

validation_accuracy = accuracy_score(Y_val, xgb_default_pred)
training_accuracy = accuracy_score(Y_train, xgb_default.predict(X_train))

print("XGBoost Untuned Training Accuracy with all features:", training_accuracy)
print("XGBoost Untuned Validation Accuracy with all features:", validation_accuracy)


In [ ]:
#Tuned XGB model
xgb_model = xgb.XGBClassifier(seed=42)


In [ ]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
 "max_depth"        : [1, 3, 4, 5, 6, 7],
 "n_estimators"     : [int(x) for x in np.linspace(start=100, stop=2000, num=10)],
 "min_child_weight" : [int(x) for x in np.arange(3, 15, 1)],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
 "subsample"        : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
 "colsample_bytree" : [0.5, 0.6, 0.7, 0.8, 0.9, 1],
 "colsample_bylevel": [0.5, 0.6, 0.7, 0.8, 0.9, 1],  
 
}

# Create an instance of GridSearchCV with the hyperparameter grid and XGBoost classifier
random_search = RandomizedSearchCV(estimator=xgb_model, 
                                   param_distributions=params, 
                                   n_iter=12, 
                                   cv=5, 
                                   n_jobs=-1, 
                                   verbose =3,
                                   random_state = 42
                )


In [ ]:
# Fit RandomizedSearchCV on the training data and Calculate tuning time
start_time = time.time()
random_search.fit(X_train_best, Y_train_best)
execution_time = time.time() - start_time

# Get the best hyperparameters and model
best_params = random_search.best_params_
best_model = random_search.best_estimator_




In [ ]:
# Evaluate the best model on the validation data
Y_pred = best_model.predict(X_val_best)
validation_accuracy = accuracy_score(Y_val_best, Y_pred)
training_accuracy = accuracy_score(Y_train_best, best_model.predict(X_train_best))

Execution_min = execution_time/60

# Print the best hyperparameters accuracy score, amd training time
print(f"Training time: {Execution_min} minutes")
print("Best hyperparameters:", best_params)
print("Training Accuracy score:", training_accuracy)
print("Validation Accuracy score:", validation_accuracy)

print(classification_report(Y_val_best, Y_pred))

In [ ]:
# Logistic model best features with tuning 
Logreg1 = LogisticRegression(max_iter=10000, random_state=seed, n_jobs=4)

# define the hyperparameters to be tuned using RandomizedSearchCV
param_distributions = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': np.linspace(0.1,0.01,1),
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'l1_ratio': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
}

# perform the random search using 5-fold cross-validation
random_search = RandomizedSearchCV(estimator=Logreg1, param_distributions=param_distributions, n_iter=12, cv=5)
random_search.fit(X_train_best, Y_train_best)

# print the best parameters and score
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

In [ ]:
# Training a Logistic Regression model based on the best parameters determined by RandomSearchCV
Logreg1 = LogisticRegression(max_iter=10000, random_state=seed, n_jobs=4, solver='newton-cg', l1_ratio=0.2, C=0.1)

Logreg1.fit(X_train_best, Y_train_best)


In [ ]:
from sklearn.metrics import mean_squared_error

print("The accuracy of the Logistic Regression Model is: " + str(round(1-mean_squared_error(Y_train_best, Logreg1.predict(X_train_best)), 3)))

In [ ]:
#SVM model best features, no tuning
svm_untunedmodel = svm.SVC(kernel = 'rbf', gamma = 0.1, random_state=42)

# Fit model to training data
svm_untunedmodel.fit(X_train_anova, Y_train_anova)

# Make predictions on test data
y_pred = svm_untunedmodel.predict(X_val_anova)

# Calculate accuracy of predictions
training_accuracy = accuracy_score(Y_train_anova, svm_untunedmodel.predict(X_train_anova))
accuracy = accuracy_score(Y_val_anova, y_pred)
print('Validation Accuracy:', accuracy)
print("Training Accuracy:", training_accuracy)

Using KNN

In [ ]:
k = 5

# Train the model
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = knn.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(Y_val, y_pred)
print("Accuracy:", accuracy)
print(classification_report(Y_val, y_pred))

In [ ]:
# On the best selected features
knn.fit(X_train_best, Y_train_best)

# Make predictions on the test set
y_pred = knn.predict(X_val_best)

# Evaluate the model
accuracy = accuracy_score(Y_val_best, y_pred)
print("Accuracy:", accuracy)
print(classification_report(Y_val_best, y_pred))

In [ ]:
# Create a KNN classifier object
knn = KNeighborsClassifier()

# Define the parameter grid to search
param_dist = {'n_neighbors': randint(1, 30),
              'p': [1, 2],
              'weights': ['uniform', 'distance']}



# Create a randomized search object
knn_random = RandomizedSearchCV(estimator = knn, 
                               param_distributions = param_dist, 
                               cv = 5, 
                               n_iter = 10, 
                               random_state=42,
                               n_jobs = -1)

In [ ]:
# Fit the randomized search object to the data
knn_random.fit(X_train, Y_train)

# Print the best parameters and the corresponding score
print("Best parameters:", knn_random.best_params_)
#print("Best score:", knn_random.best_score_)
# Evaluate the model
knn_random_pred = knn_random.predict(X_val)
training_accuracy = accuracy_score(Y_train, knn_random.predict(X_train))
accuracy = accuracy_score(Y_val, knn_random_pred)
print("Validation Accuracy:", accuracy)
print('Training Accuracy:', training_accuracy)


In [ ]:
# On best features
# Fit the randomized search object to the data
knn_random.fit(X_train_best, Y_train_best)

# Print the best parameters and the corresponding score
print("Best parameters:", knn_random.best_params_)
#print("Best score:", knn_random.best_score_)

knn_random_pred = knn_random.predict(X_val_best)
# Evaluate the model
training_accuracy = accuracy_score(Y_train_best, knn_random.predict(X_train_best))
accuracy = accuracy_score(Y_val_best, knn_random_pred)
print("Validation Accuracy:", accuracy)
print('Training Accuracy:', training_accuracy)
print(classification_report(Y_val_best, knn_random_pred))


Using the MLP NN

In [ ]:
modelsd = Sequential()

modelsd.add(Dense(units=18, activation="ReLU", input_shape = (18,)))

modelsd.add(Dense(units=200, activation="ReLU"))

modelsd.add(Dense(units=1000, activation="ReLU"))

modelsd.add(Dense(units=1000, activation="ReLU"))

modelsd.add(Dense(units=810, activation="ReLU"))
modelsd.add(Flatten())
modelsd.add(Dense(units=2, activation="softmax"))


modelsd.compile(
    optimizer = "adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
modelsd.fit(X_train_best, Y_train_best, validation_data=(X_val_best,Y_val_best), batch_size = 160, epochs=250, shuffle=True)


In [ ]:
modelsd_pred = modelsd.predict(X_val_best)
modelsd.evaluate(X_val_best, Y_val_best)


MODEL PIPELINE OF MLP-XGBOOST

In [ ]:
# Using the tuned MLP with best hyperparameters to extract features
mlp = Sequential()

mlp.add(Dense(units=18, activation="ReLU", input_shape = (18,)))
#from tensorflow.keras.layers import Dropout


mlp.add(Dense(units=200, activation="ReLU"))

mlp.add(Dense(units=1000, activation="ReLU"))

mlp.add(Dense(units=1000, activation="ReLU"))

mlp.add(Dense(units=810, activation="ReLU"))
mlp.add(Flatten())
mlp.add(Dense(units=2, activation="softmax"))


mlp.compile(
    optimizer = "adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)


#Compile and fit the model to training data
mlp.fit(X_train_best, Y_train_best, batch_size = 160, epochs=250, shuffle=True)

# Extract features from the labeled data using the MLP
X_train_features = mlp.predict(X_train_best)

# Train the XGBoost model with best hyperparametrs on the learned features
final_xgb_model = xgb.XGBClassifier(subsample=1, n_estimators=1366, min_child_weight=6, max_depth=5, learning_rate=0.15, gamma=0.0, colsample_bytree=0.6, colsample_bylevel= 0.6, seed=42)
final_xgb_model.fit(X_train_features, Y_train_best)

In [ ]:
# threshold = 0.5

# binary_preds = np.where(modelsd_pred[:, 1] >= threshold, 1, 0)
# sd = Y_val_best.values

# print(classification_report(Y_val, y_pred))

In [ ]:
# Extract features from the labeled data using the MLP
X_val_features = mlp.predict(X_val_best)

Combined_pred = final_xgb_model.predict(X_val_features)
Train_accuracy = accuracy_score(Y_train_best, final_xgb_model.predict(X_train_features))
Valid_accuracy = accuracy_score(Y_val_best, Combined_pred)
print("MLP-XGB Validation Accuracy:", Valid_accuracy)
print('MLP-XGB Training Accuracy:', Train_accuracy)
print(classification_report(Y_val_best, Combined_pred))

Best Model: Tuned XGBoost Model by itself
The best performing model is the tuned XGBoost, we wil be using this to make predictions on the test_data and creating a group csv file

In [ ]:
# Load the test data
New_data = pd.read_csv("/content/drive/MyDrive/Hackathon/Data/test_data.csv")
New_copy = New_data.copy()



In [ ]:
#Preprocess Data
# One hot encode categorical data
New_copy = pd.get_dummies(New_copy, columns=['RoomType', 'MealPlan', 'MarketSegment'])
New_copy = New_copy.rename(columns={'ArrivalYear': 'year', 'ArrivalMonth': 'month', 'ArrivalDate': 'day'})

In [ ]:
# Extract best 18 X features for predictions
Features = New_copy.drop(['BookingID'], axis=1)
Features = Features.loc[:, selected_features]

In [ ]:
New_copy.head()

In [ ]:
# Function to convert binary to string values for our predicted column
def bintostr(x):
    if x == 1:
      x = 'Canceled'
    elif x == 0:
      x = 'Not_Canceled'
    return x

In [ ]:
# Predict the cancelations
predicted_values = best_model.predict(Features)

# Put them in the original dataframe
New_data['BookingStatus'] = predicted_values
New_data['BookingStatus'] = New_data['BookingStatus'].map(bintostr)

In [ ]:
# Save the modified DataFrame back to a CSV file
New_data.to_csv('group_15.CSV', index=False)